<a href="https://colab.research.google.com/github/AnshChoudhary/3D-Model-Website-Car-/blob/main/Ansh_Choudhary_(tested)_CAMB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing the necessary Librarys

In [2]:
%pip install youtube_search
%pip install pytube
%pip install youtube_transcript_api
%pip install pydub
%pip install SpeechRecognition
%pip install librosa
!apt-get install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 33.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


# Data Collection:
 Searching the Irish YouTube channel and Downloading the latest 2 videos from it.

 Extracting the audio and transcripts from the video.

In [3]:
from pytube import YouTube
import os
from youtube_search import YoutubeSearch
from youtube_transcript_api import YouTubeTranscriptApi

def search_and_download_irish_videos(query, output_path, max_results=10):
    # Perform YouTube search using the query
    results = YoutubeSearch(query, max_results=max_results).to_dict()

    # Create folders for audio files and transcripts
    audio_folder = os.path.join(output_path, "audio")
    transcript_folder = os.path.join(output_path, "transcripts")
    os.makedirs(audio_folder, exist_ok=True)
    os.makedirs(transcript_folder, exist_ok=True)

    # Iterate over search results
    for video in results:
        video_id = video['id']
        video_title = video['title']

        try:
            # Download video
            yt = YouTube(f'https://www.youtube.com/watch?v={video_id}')
            stream = yt.streams.filter(file_extension='mp4').first()
            video_file = stream.download(output_path=output_path, filename=f'{video_id}')
            print(f"Downloaded video: {video_title}")

            # Download audio
            stream_audio = yt.streams.filter(only_audio=True).first()
            audio_file = stream_audio.download(output_path=audio_folder, filename=f'{video_id}.wav')
            print(f"Downloaded audio: {video_title}")

            # Download transcript
            transcript = YouTubeTranscriptApi.get_transcript(video_id)
            transcript_text = ' '.join([line['text'] for line in transcript])
            transcript_file = os.path.join(transcript_folder, f'{video_id}.txt')
            with open(transcript_file, 'w', encoding='utf-8') as f:
                f.write(transcript_text)
            print(f"Downloaded transcript: {video_title}")

        except Exception as e:
            print(f"Error downloading {video_title}: {e}")

# Example usage:
query = "Gaeilge i mo chroí"
output_path = "/content/files"
search_and_download_irish_videos(query, output_path, max_results=2)


Downloaded video: A casual conversation as Gaeilge with Gaeilge le Jane 🥰 💚 MUNSTER DIALECT, Gaeltacht Mhúscraí
Downloaded audio: A casual conversation as Gaeilge with Gaeilge le Jane 🥰 💚 MUNSTER DIALECT, Gaeltacht Mhúscraí
Downloaded transcript: A casual conversation as Gaeilge with Gaeilge le Jane 🥰 💚 MUNSTER DIALECT, Gaeltacht Mhúscraí
Downloaded video: How to start speaking Irish | Gaeilge i Mo Chroí
Downloaded audio: How to start speaking Irish | Gaeilge i Mo Chroí
Downloaded transcript: How to start speaking Irish | Gaeilge i Mo Chroí


In [ ]:
# import os

# # Set the path to your service account key file
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/potent-cedar-415705-417bcf87622a.json"

# Data Preprocessing:
 Encoding the audio files.

In [4]:
import os


audio_folder = '/content/files/audio'


output_folder = '/content/files/encoded audio'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Iterate over each file in the audio folder
for filename in os.listdir(audio_folder):
    if filename.endswith('.wav'):
        # Generate input and output file paths
        input_file = os.path.join(audio_folder, filename)
        output_file = os.path.join(output_folder, filename)

        # Execute FFmpeg command to re-encode the audio file
        !ffmpeg -i "$input_file" -c:a pcm_s16le "$output_file"


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Auto align the audio and the transcript in a folder .

In [5]:
import os
import shutil

def align_audio_transcripts(audio_folder, transcript_folder, output_folder):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get list of audio files
    audio_files = os.listdir(audio_folder)

    # Iterate over audio files
    for audio_file in audio_files:
        # Construct paths for audio and transcript files
        audio_path = os.path.join(audio_folder, audio_file)
        transcript_file = os.path.splitext(audio_file)[0] + '.txt'
        transcript_path = os.path.join(transcript_folder, transcript_file)

        # Check if transcript file exists
        if not os.path.exists(transcript_path):
            print(f"Transcript file not found for {audio_file}")
            continue

        # Construct output paths
        output_audio_path = os.path.join(output_folder, audio_file)
        output_transcript_path = os.path.join(output_folder, transcript_file)

        # Copy audio file to output folder
        shutil.copy(audio_path, output_audio_path)

        # Copy transcript file to output folder
        shutil.copy(transcript_path, output_transcript_path)

        print(f"Aligned {audio_file} and {transcript_file} saved in {output_folder}")

# Specify input and output folders
audio_folder = '/content/files/encoded audio'
transcript_folder = '/content/files/transcripts'
output_folder = '/content/files/aligned data'

# Call function to align audio and transcripts
align_audio_transcripts(audio_folder, transcript_folder, output_folder)


Aligned ViGAb66Nsdo.wav and ViGAb66Nsdo.txt saved in /content/files/aligned data
Aligned zqWQz8rOk98.wav and zqWQz8rOk98.txt saved in /content/files/aligned data


In [9]:
from IPython.display import Audio, display

sound_file = '/content/files/aligned data/ViGAb66Nsdo.wav'

display(Audio(sound_file, autoplay=True))

# Data Augmentation

In [10]:
from pydub import AudioSegment
from pydub.effects import speedup
import os

def augment_audio(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".wav"):
            input_file = os.path.join(input_folder, filename)
            output_file = os.path.join(output_folder, filename)

            # Load the audio file
            audio = AudioSegment.from_wav(input_file)

            # Apply speedup effect to the audio
            augmented_audio = speedup(audio, playback_speed=1.5)

            # Export the augmented audio to a new file
            augmented_audio.export(output_file, format="wav")
            print(f"Augmented audio saved as: {output_file}")

# Specify input and output folders
input_folder = "/content/files/encoded audio"
output_folder = "/content/files/augmented_audio"

# Augment audio files
augment_audio(input_folder, output_folder)


Augmented audio saved as: /content/files/augmented_audio/ViGAb66Nsdo.wav
Augmented audio saved as: /content/files/augmented_audio/zqWQz8rOk98.wav


In [13]:
from IPython.display import Audio, display

sound_file = '/content/files/augmented_audio/ViGAb66Nsdo.wav'

display(Audio(sound_file, autoplay=True))

# Dataset Creation

Run a frequency test to check if a speaker is male or female.(It may not be the prefect way but can be considered)

In [11]:
import librosa
import numpy as np

# Function to extract pitch from audio using Fourier Transform
def extract_pitch(audio_file):
    y, sr = librosa.load(audio_file)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch_mean = np.mean(pitches[np.nonzero(pitches)]) # Mean of non-zero pitch values
    return pitch_mean

# Function to classify voice as male or female based on pitch threshold
def classify_voice(pitch):
    # Define threshold values for male and female voices
    male_threshold = 120  # Adjust threshold as needed
    female_threshold = 220  # Adjust threshold as needed

    if pitch < male_threshold:
        return "Male"
    elif pitch > female_threshold:
        return "Female"
    else:
        return "Unknown"

# Test the functions
audio_file = "/content/files/audio/ViGAb66Nsdo.wav"  # Path to the WAV audio file
pitch = extract_pitch(audio_file)
gender = classify_voice(pitch)
print("Detected voice gender:", gender)


<ipython-input-11-80d1d825542f>:6: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Detected voice gender: Female


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# prompt: write code to download all the files and folders in the files folder

!zip -r /content/drive/MyDrive/files.zip /content/files


  adding: content/files/ (stored 0%)
  adding: content/files/audio/ (stored 0%)
  adding: content/files/audio/ViGAb66Nsdo.wav (deflated 2%)
  adding: content/files/audio/zqWQz8rOk98.wav (deflated 2%)
  adding: content/files/ViGAb66Nsdo (deflated 1%)
  adding: content/files/encoded audio/ (stored 0%)
  adding: content/files/encoded audio/ViGAb66Nsdo.wav (deflated 28%)
  adding: content/files/encoded audio/zqWQz8rOk98.wav (deflated 43%)
  adding: content/files/zqWQz8rOk98 (deflated 2%)
  adding: content/files/aligned data/ (stored 0%)
  adding: content/files/aligned data/ViGAb66Nsdo.txt (deflated 59%)
  adding: content/files/aligned data/zqWQz8rOk98.txt (deflated 48%)
  adding: content/files/aligned data/ViGAb66Nsdo.wav (deflated 28%)
  adding: content/files/aligned data/zqWQz8rOk98.wav (deflated 43%)
  adding: content/files/augmented_audio/ (stored 0%)
  adding: content/files/augmented_audio/ViGAb66Nsdo.wav (deflated 28%)
  adding: content/files/augmented_audio/zqWQz8rOk98.wav (deflated